# Initial Data Checks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
from astropy.modeling import models, fitting

<div class='alert alert-info'>
    <font size='3'><b>These cells contain instructions. Edit the paths below to select a rate file.</b></font>
</div>

In [ ]:
## path to segments of grism rate file
gr_rate = 'ex_rate/jw01442001001_01101_00002-seg003_nrca5_rate.fits' ## path to a grism rate file
im_rate = 'ex_rate/jw01442001001_01101_00001-seg003_nrca3_rate.fits' ## path to an imaging rate file

#gr_rate = '../pipe_process/ex_proc_data/jw01442001001_01101_00002_nrca5_uncal_sat_rate.fits'
#im_rate = '../pipe_process/ex_proc_data/jw01442001001_01101_00001_nrca3_saturated_rate.fits'

# Example where the source is shifted and at lower flux (such as a misidentified source)
#gr_rate = '../pipe_process/ex_proc_data/jw01442001001_01101_00002_nrca5_uncal_faint_and_shifted_rate.fits'
#im_rate = '../pipe_process/ex_proc_data/jw01442001001_01101_00001_nrca3_shifted_and_fainter_rate.fits'


<div class='alert alert-info'>
    <font size='3'><b>Edit the approximate count rate from the ETC.</b></font>
</div>

In [ ]:
gainA5 = 1.8 ## e-/DN
exp_grism_rate = 2540. / gainA5 ## DN/s

gainSW = 2.17 ## e-/DN for A3
#gainSW = 2.08 ## e-/DN for A1
exp_im_rate = 6.0e6 / gainSW ## DN/s, src=2 arcsec, backg=2.1 to 2.5 arcsec


## Make sure the Spectrum is in the Right Place

In [ ]:
specdat = fits.getdata(gr_rate)

In [ ]:
fig, ax = plt.subplots(figsize=(12,3))
ax.imshow(specdat,vmin=0,vmax=500,origin='lower')
ax.set_xlabel("Y Pixel")
fig, ax = plt.subplots(figsize=(12,3))
ax.imshow(specdat,vmin=0,vmax=500,origin='lower')
ax.set_xlabel("X Pixel")
ax.set_xlim(500,1050)

In [ ]:
profile = np.mean(specdat[:,512:1024],axis=1)
x = np.arange(specdat.shape[0])
#plt.plot(x,profile)

In [ ]:

# Fit the data using a Gaussian
g_init = models.Gaussian1D(amplitude=200., mean=34, stddev=2.)
fit_g = fitting.LevMarLSQFitter()
g = fit_g(g_init, x, profile)

# Plot the data with the best-fit model
plt.figure(figsize=(8,5))
plt.plot(x, profile, 'ko')
#plt.plot(x, t(x), label='Trapezoid')
plt.plot(x, g(x), label='Gaussian Fit')
plt.xlabel('Position')
plt.ylabel('Flux')
plt.legend(loc=2)

In [ ]:
g.mean

<div class='alert alert-warning'>
    <font size='3'><b>Check: Does the source land near Y=33 (zero-based)?</b></font>
</div>

## Do a crude extraction to check counts

In [ ]:
ycen = int(np.round(g.mean.value))
ylow, yhigh = ycen - 2, ycen + 3

In [ ]:
quick_extract = np.sum(specdat[ylow:yhigh,:],axis=0)

In [ ]:
fig, ax = plt.subplots()
ax.plot(quick_extract)
ax.set_ylim(0,2. * exp_grism_rate)
ax.axhline(exp_grism_rate,color='red',label='Approx Expected Level')
ax.legend()
ax.set_xlabel("X Pixel")
ax.set_ylabel("Extracted Flux (DN/s)")

## Is it within a factor of 4?

In [ ]:
fratio = quick_extract/exp_grism_rate

In [ ]:
fig, ax = plt.subplots()
ax.plot(fratio)
ax.axhline(0.25,color='red')
ax.axhline(4.0,color='red')

<div class='alert alert-warning'>
    <font size='3'><b>Check: Is the source within a factor 4 of expectations?</b></font>
</div>

## Check for saturation

In [ ]:
HDUList = fits.open(gr_rate)

In [ ]:
dq = HDUList['DQ'].data

In [ ]:
sat = (dq & (1 << 1) > 0) ## test if bit 1 (zero-based) is True

In [ ]:
fig, ax = plt.subplots(figsize=(12,3))
ax.imshow(sat,origin='lower')
fig, ax = plt.subplots(figsize=(12,3))
ax.imshow(sat,origin='lower')
ax.set_xlim(500,1050)

<div class='alert alert-warning'>
    <font size='3'><b>Check: Is the source saturated?</b></font>
</div>

## Make sure the Image is in the Right Place

In [ ]:
imdat = fits.getdata(im_rate)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(imdat,vmin=0,vmax=200,origin='lower')
ax.set_xlabel("X Pixel")
ax.set_ylabel("Y Pixel")

ax.set_xlim(950,1150)

<div class='alert alert-warning'>
    <font size='3'><b>Check: Is the source near the middle?</b></font>
</div>

## Crude Photometry

In [ ]:
simplePhot = np.sum(imdat[0:128,990:1110])

In [ ]:
simplePhot

In [ ]:
exp_im_rate

In [ ]:
simplePhot / exp_im_rate

<div class='alert alert-warning'>
    <font size='3'><b>Check: Is the source within a factor of 4 of expectations?</b></font>
</div>

## Check for saturation

In [ ]:
im_dq = fits.getdata(im_rate,extname='DQ')
im_sat = (im_dq & (1 << 1)) > 0 ## test if bit 1 (zero-based) is True

In [ ]:
fig, ax = plt.subplots()
ax.imshow(im_sat,origin='lower')
ax.set_xlabel("X Pixel")
ax.set_ylabel("Y Pixel")

ax.set_xlim(950,1150)

<div class='alert alert-warning'>
    <font size='3'><b>Check: Is the source saturated?</b></font>
</div>